# Introduction

In the following notebook, I will be performing an EDA of some Airbnb data in the San Francisco area. This data pertains to the last calendar year, which at the time of this analysis would be December 2018 - December 2019.

In the following analysis, I will be looking to explore the data and answer the following questions:


- What are some of the seasonal Airbnb trends you've noticed in the San Francisco area? When are the down months over the last year? Which months are the most busy? Which areas were the most popular during these times? How have prices changed over time?
- Are airbnb's that are available for monthly rent competitive with the local markets?


*GitHub Repo References*
The raw data files can be found [here](https://github.com/KishenSharma6/Airbnb-SF_ML_-_Text_Analysis/tree/master/Data/01_Raw).

The raw data aggregation scripts can be found [here](https://github.com/KishenSharma6/Airbnb-SF_ML_-_Text_Analysis/tree/master/Project%20Codes/01.%20Raw%20Data%20Aggregation%20Scripts).

The data cleaning scripts used to tidy the raw data aggregation can be found [here](https://github.com/KishenSharma6/Airbnb-SF_ML_-_Text_Analysis/tree/master/Project%20Codes/02.%20Data%20Cleaning%20Scripts).

In [ ]:
#Read in libraries
import dask.dataframe as dd
import swifter

import pandas as pd
import pandas_profiling

import re

import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#supress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#Set plot aesthetics for notebook
sns.set(style='whitegrid', palette='pastel', color_codes=True)

#Increase number of columns and rows displayed by Pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

#Set float format
pd.options.display.float_format = '{:.0f}'.format

#Ignore warnings
import warnings; warnings.simplefilter('ignore')

**Read in Data**

In [ ]:
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate/'

In [ ]:
#Read in Airbnb Listings Data
listings = pd.read_csv(path + '01_04_2020_Listings_Cleaned.csv',index_col=0, low_memory=True, sep=',')

#Read in Airbnb Calendar and Reviews data
calendar = pd.read_csv(path + '01_04_2020_Calendar_Cleaned.csv', sep = ',',
                       parse_dates=['date'], low_memory=True,index_col=0)

In [ ]:
#Read in Zillow data
zillow = pd.read_csv(path + '12_29_2019_Zillow_Cleaned.csv', parse_dates=['Date'],
                     index_col=0, sep=',')

# Zillow

#### How has Airbnb grown over the last year(12/2018 - 12/2019)?

Let's begin by taking a look at the number of unique listings per day, week, and month available for rent from Airbnb hosts.

In [ ]:
calendar['month_year'] = pd.to_datetime(calendar['date']).dt.to_period('M')
#Set date as index for calendar data
calendar.set_index('date', inplace=True)

#Capture appropriate dates for analysis in Calendar data
calendar= calendar.loc['2018-12-01':'2020-01-01']

# #Convert calendar to dask
# calendar = dd.from_pandas(calendar, npartitions=3)

#Extract date info from index and assign as columns
calendar['year'] = calendar.index.year
calendar['month'] = calendar.index.month
calendar['weekday'] = calendar.index.weekday_name

#Check
calendar.head()

In [ ]:
#Group last year by month_year and get a count of unique listings per day
last_year_daily = calendar.groupby(['date'])['listing_id'].agg({'nunique'}).reset_index()

#Rename columns
last_year_daily =last_year_daily.rename(columns = {'nunique': 'listings'})

In [ ]:
#Weekly listings
print(last_year_daily.shape)
last_year_daily.head()

In [ ]:
rolling_mean=last_year_daily.rolling(window=7,min_periods=1).mean()
rolling_mean = rolling_mean.rename(columns={'listings':'rolling_wk_avg'})

In [ ]:
rolling_mean.head()

In [ ]:
test= last_year_daily.join(rolling_mean)

In [ ]:
test.head()

In [ ]:
test = test.drop(columns='listings')

In [ ]:
#Capture unique listings a month
last_year_monthly = calendar.groupby(['month_year'])['listing_id'].agg({'nunique'}).reset_index()

#Rename columns
last_year_monthly =last_year_monthly.rename(columns = {'nunique': 'listings'})

In [ ]:
#Set 538 plot style
plt.style.use('fivethirtyeight')

#Plot daily unique listings data
ax = last_year_daily.plot(x='date', y='listings', kind = 'line',style='o-', markersize=.5,
          label= 'Daily Unique Listings',figsize = (15,10),
         linewidth = 1.5)

test.plot(x='date', y='rolling_wk_avg', kind = 'line',style='o-', markersize=.5,
          label= 'Weekly Unique Listings',figsize = (15,10),
         linewidth = 1.5, ax=ax)

#plot montly average
last_year_monthly.plot(x='month_year', y='listings',kind = 'line',style='o-', markersize= 7,
          label= 'Monthly Unique Listings', linewidth = 1.5 , ax=ax)

#Set fontdict
fontdict={'weight' : 'bold',
          'size': 17}

#Set x and y labels
ax.set_xlabel('Month',fontdict=fontdict)
ax.set_ylabel('Count', fontdict=fontdict)

#Format yticks
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])

#Set Title
ax.set_title('Growth of Airbnb Listings in San Francisco', fontweight = 'bold', fontsize=22)

#Adjust plot margins
ax.margins(0,.12)

#Mute vertical grid lines
ax.grid(b = False, which ='major', axis = 'x')

#Add Text
xs,ys=last_year_monthly['month_year'], last_year_monthly['listings']

for x,y in zip(xs,ys):
    label = '{:,}'.format(y)
    plt.annotate(label, (x,y),textcoords="offset points",fontsize = 9, xytext=(0,0), ha='left')

#Set legend
plt.legend(title='Legend', frameon = True, loc='upper left');

Over the last year, there has been significant growth in the number of listings available for rent month to month. Let us look into the number of nights booked by users over the last year.

In [ ]:
#Set Date as index for zillow data
zillow.set_index(keys = 'Date', inplace=True)

In [ ]:
#Extract date info from index and assign to zillow
zillow['Year'] = zillow.index.year
zillow['Month'] = zillow.index.month
zillow['Weekday_Name'] = zillow.index.weekday_name

In [ ]:
zillow.head()

# Time Series Analysis

For this analysis, we will isolate the rows in our Zillow data that are in the zipcode or city of the listings data.

In [ ]:
#Capture zip information from listings
listings_zip = list(listings.zipcode.unique())

#Capture city information from listings 
listings_cities = list(listings.city.unique())

#Capture SF Data
sf_zillow = zillow[zillow['Zip'].isin(listings_zip) | zillow['City'].isin(listings_cities)]

#Check
print(sf_zillow.shape)
display(sf_zillow.tail())

In [ ]:
zillow = zillow.loc['2011-09-01':'2019-11-01']

In [ ]:
#SF vs rest of US Rent

plt.style.use('ggplot')

fig, ax = plt.subplots(figsize = (12,5))
sf_zillow.groupby('Date')['Median_Rent'].mean().plot(kind = 'line', color = 'r')

zillow.groupby(['Date'])['Median_Rent'].mean().plot(kind = 'line', color= 'g')#natl average

plt.title('Comparing rent accross the Unites states vs SF(in airbnb specofically)')
#We also noted each state's region size ranking, which represents how big it is population-wise;
#California is ranked No. 1 with the largest population of all states, while Wyoming is ranked No. 51
zillow[zillow.SizeRank < 2].groupby(['Date'])['Median_Rent'].mean().plot(kind = 'line', color = 'b')#this is compared to CA
zillow[zillow.SizeRank < 10].groupby(['Date'])['Median_Rent'].mean().plot(kind = 'line', color = 'black')#purp(top 10 largest in US)

Airbnb monthly rent vs bay area rent 

In [ ]:
#Capture id and monthly_price from listings data for merge with calendar data
listings_merge = listings[['id','monthly_price']]



In [ ]:
calendar_merge = calendar[['listing_id','price','month_year','year','month']]
calendar_merge.reset_index(inplace = True)

In [ ]:
monthly_listings = calendar_merge.merge(listings_merge, left_on='listing_id', right_on='id')

In [ ]:
monthly_listings=monthly_listings[-monthly_listings.monthly_price.isna()]

monthly_listings.drop_duplicates(inplace = True)

In [ ]:
print(monthly_listings.shape)
monthly_listings.head()

In [ ]:
#Convert date to month-year format
test_pd
#test_pd['date'] = pd.to_datetime(test_pd['date']).dt.to_period('M')

In [ ]:
test_pd.set_index('date', inplace=True)

In [ ]:
test_pd.sort_index(inplace=True)

In [ ]:
test_pd['Year'] = test_pd.index.year
test_pd['Month'] = test_pd.index.month
test_pd['Weekday Name'] = test_pd.index.weekday_name

In [ ]:
test_pd.head()

In [ ]:
test_pd.loc['2018-09':'2019-12'].groupby('date')['monthly_price'].mean().plot()

In [ ]:
sf_zillow.loc['2018-09':'2019-12'].groupby(['Date'])['Median_Rent'].mean().plot(kind = 'line')

In [ ]:
#test_pd.loc['2018-09':'2019-12'].groupby('date')['monthly_price'].mean().plot(kind = 'box')
sns.boxplot(x='Month', y = 'monthly_price', data = test_pd)

In [ ]:
sf_zillow.tail()

In [ ]:
calendar.head()

* Comparing monthly rent of airbnb to zillow

### Principal Component Analysis